## Задача классификации 
### По имеющимуся датасету обучить модель распозновать фамилии

![title](pik/1.png)

In [124]:
# импортируем нужные библиотеки
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
from sklearn.linear_model import LogisticRegression 
import nltk
from nltk.tokenize import word_tokenize

from IPython.core.display import display, HTML # расширяем блокнот
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns', None) # выводим все колонки

from IPython.core.interactiveshell import InteractiveShell # выводим все результаты из ячейки
InteractiveShell.ast_node_interactivity = "all"

In [125]:
# загружаем данные 
dtrain = pd.read_csv('data/linear_train.txt', sep=',', header=None)
dtest = pd.read_csv('data/linear_test.txt', header=None)
dtrain.columns = ['surname','answer']
dtest.columns = ['surname']

In [126]:
df = pd.concat([dtrain['surname'], dtest['surname']])
y = dtrain['answer']

In [50]:
def extractDigits(lst): 
    return [[el] for el in lst] 

In [51]:
lst = df
nested = extractDigits(lst)

In [52]:
def apply_nested(func, lst, isatom=lambda item: not isinstance(item, list)):
    stack = [lst]
    while stack:
        lst = stack.pop()
        for i, item in enumerate(lst):
            if isatom(item):
                lst[i] = func(item)
            else:
                stack.append(item)

In [53]:
apply_nested(lambda atom: ','.join(atom), nested)

In [54]:
dff = pd.DataFrame(nested)

In [56]:
dff.columns = ['surname']

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
tfidf = TfidfVectorizer(analyzer = 'char' )
X_train_tf = tfidf.fit_transform(dff['surname'][:101408])
X_test_tf =  tfidf.transform(dff['surname'][101408:])

In [61]:
print(X_train_tf.shape)
print(X_test_tf.shape)

(101408, 52)
(188920, 52)


In [63]:
tfidf.vocabulary_.get(u'algoritm')
tfidf_transformer = TfidfTransformer()
X_train = tfidf_transformer.fit_transform(X_train_tf)
X_test = tfidf_transformer.transform(X_test_tf)

In [123]:
print(X_train.shape)
print(X_test.shape)
print(y.shape)

(101408, 52)
(188920, 52)
(101408,)


## Обучаем модель

![title](pik/2.png)

In [66]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn import model_selection, datasets, linear_model, metrics

# логистическая регрессия

In [68]:
from sklearn.linear_model import LogisticRegression
modelLR = LogisticRegression(
    random_state=42,
    # метод для поиска решения. Для небольших датасетов лучше подходит liblinear, sag и saga -- для больших.
    # Варианты: newton-cg, lbfgs, liblinear, sag, saga
    #solver='sag',
    # норма для регуляризации. Варианты: l2, l1.
    #penalty='l2',
    # параметр регуляризации. Чем меньше, тем сильнее регуляризация. Положительный.
    #C=1,
    # параметр для остановки поиска решения.
    #tol=1e-4,
    # Так как распознается 2 класса.
    #multi_class='ovr',
    class_weight='balanced'
    )

In [69]:
modelLR.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [70]:
parameters_grid_LR = {
    'penalty' : ['l1', 'l2'],
    'C' : np.logspace(0.00001, 0.01, 3, 10),
    'max_iter' : np.arange(5,10),
    'class_weight' : ['balanced'],
    'solver' : ['lbfgs', 'liblanear', 'sag', 'saga']
}

In [71]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

In [72]:
grid_cv_LR = model_selection.GridSearchCV(modelLR, parameters_grid_LR, scoring = 'accuracy', cv = cv)

In [73]:
%%time
grid_cv_LR.fit(X_train, y)

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got liblanear.

  FitFailedWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver sag supports only 'l

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got liblanear.

  FitFailedWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/s

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scik

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

...

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalt

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got liblanear.

  FitFailedWarning)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/s

CPU times: user 3min 34s, sys: 4.84 s, total: 3min 39s
Wall time: 3min 50s


/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.00002303, 1.0115911 , 1.02329299]),
                         'class_weight': ['balanced'],
                         'max_iter': array([5, 6, 7, 8, 9]),
                         'penalty': ['l1', 'l2']

In [74]:
grid_cv_LR.best_estimator_

LogisticRegression(C=1.0000230261160268, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=6, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
grid_cv_LR.best_score_

0.7074598165861354

In [76]:
grid_cv_LR.best_params_

{'C': 1.0000230261160268,
 'class_weight': 'balanced',
 'max_iter': 6,
 'penalty': 'l2',
 'solver': 'saga'}

In [77]:
predsLR = grid_cv_LR.predict(X_test)

## деревья решений

In [78]:
from sklearn.tree import DecisionTreeClassifier
modelDTC = DecisionTreeClassifier(
    class_weight = 'balanced'
)

In [79]:
modelDTC.get_params()

{'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [80]:
parameters_grid_DTC = {
    'max_leaf_nodes': [2, 50, 100], 
    'min_samples_split': [2, 3, 4],
    'class_weight' : ['balanced']
}

In [81]:
grid_cv_DTC = model_selection.GridSearchCV(modelDTC, parameters_grid_DTC, scoring = 'accuracy', cv = cv)

In [82]:
%%time
grid_cv_DTC.fit(X_train, y)

CPU times: user 2min 13s, sys: 2.71 s, total: 2min 16s
Wall time: 2min 29s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                              class_weight='balanced',
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                           

In [83]:
grid_cv_DTC.best_score_

0.6574746080268217

In [84]:
grid_cv_DTC.best_params_

{'class_weight': 'balanced', 'max_leaf_nodes': 100, 'min_samples_split': 2}

In [85]:
predsDTC = grid_cv_DTC.predict(X_test)

## случайный лес

In [86]:
from sklearn.ensemble import RandomForestClassifier
modelRFC = RandomForestClassifier(
    class_weight = 'balanced'
)

In [87]:
modelRFC.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [88]:
parameters_grid_RFC = {
    #'bootstrap': [True],
    #'max_depth': [80, 100, 120],
    #'max_features': [2, 3],
    #'min_samples_leaf': [3, 4, 5],
    #'min_samples_split': [8, 10, 12],
    #'n_estimators': [40],
    'class_weight' : ['balanced']
}

In [89]:
grid_cv_RFC = model_selection.GridSearchCV(modelRFC, parameters_grid_RFC, scoring = 'accuracy', cv = cv)

In [90]:
%%time
modelRFC.fit(X_train, y)

CPU times: user 4min 5s, sys: 3.9 s, total: 4min 9s
Wall time: 4min 18s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [91]:
predsRFC = modelRFC.predict(X_test)

## SGD

In [92]:
modelSGD = linear_model.SGDClassifier(random_state = 0, tol=1e-3, class_weight = 'balanced')

In [93]:
modelSGD.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': 'balanced',
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'hinge',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': 0,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [94]:
parameters_grid_SGD = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
    'class_weight' : ['balanced']
}

In [95]:
grid_cv_SGD = model_selection.GridSearchCV(modelSGD, parameters_grid_SGD, scoring = 'accuracy', cv = cv)

In [96]:
%%time
grid_cv_SGD.fit(X_train, y)

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


CPU times: user 7min 53s, sys: 13.4 s, total: 8min 6s
Wall time: 9min 2s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight='balanced',
                                     early_stopping=False, epsilon=0.1,
                                     eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penal...
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'class_weight': ['balanced'],
          

In [97]:
grid_cv_SGD.best_estimator_

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=9,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=0, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [98]:
grid_cv_SGD.best_score_

0.6872054038063308

In [99]:
grid_cv_SGD.best_params_

{'alpha': 0.0001,
 'class_weight': 'balanced',
 'loss': 'log',
 'max_iter': 9,
 'penalty': 'l1'}

In [100]:
preds_SGD = grid_cv_SGD.predict(X_test)

## Cтекинг

In [101]:
from sklearn.ensemble import  StackingClassifier

In [102]:
base_learners = [
                 ('rf_1', RandomForestClassifier(
                     class_weight = 'balanced')),
                 ('rf_2', linear_model.SGDClassifier(
                     class_weight = 'balanced',)),
                 ('rf_3', DecisionTreeClassifier(
                     class_weight = 'balanced')),
                 ('rf_4', LogisticRegression(
                     class_weight = 'balanced',)),
                ]

In [117]:
f_estimator=LogisticRegression(class_weight = 'balanced')

In [118]:
clf = StackingClassifier(estimators=base_learners, final_estimator=f_estimator, cv=5)

In [119]:
%%time
clf.fit(X_train, y)

/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/antonskarednov/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

CPU times: user 22min 26s, sys: 16.5 s, total: 22min 42s
Wall time: 24min 11s


StackingClassifier(cv=5,
                   estimators=[('rf_1',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight='balanced',
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                

In [120]:
from sklearn.metrics import classification_report
y_pred_CLF = clf.predict(X_train)
classification_report(y, y_pred_CLF)

'              precision    recall  f1-score   support\n\n           0       0.99      0.70      0.82     90770\n           1       0.27      0.94      0.42     10638\n\n    accuracy                           0.73    101408\n   macro avg       0.63      0.82      0.62    101408\nweighted avg       0.92      0.73      0.78    101408\n'

In [121]:
from sklearn import metrics
conf_matCLF = metrics.confusion_matrix(y, y_pred_CLF)
conf_matCLF = pd.DataFrame(conf_matCLF, index=clf.classes_, columns=clf.classes_)
conf_matCLF

,0,1
0,63664,27106
1,597,10041


In [122]:
preds_clf = clf.predict(X_test)

In [111]:
ot_clf = pd.DataFrame(q, columns=['Id'])

In [112]:
ot_clf['Answer'] = preds_clf

In [113]:
ot_clf

,Id,Answer
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
188915,188915,0
188916,188916,0
188917,188917,0
188918,188918,0


In [123]:
ot_clf.to_csv('ot_clf.csv', index=False)